In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
 !ls

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Lambda,Concatenate
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D, Add
from keras.models import Model
from keras import regularizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import glorot_normal
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [0]:
import tensorflow as tf

In [0]:
def two_path(X_input):
  
  X = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
  X = BatchNormalization()(X)
  X1 = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
  X1 = BatchNormalization()(X1)
  X = layers.Maximum()([X,X1])
  X = Conv2D(64,(4,4),strides=(1,1),padding='valid',activation='relu')(X)
  
  X2 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
  X2 = BatchNormalization()(X2)
  X21 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
  X21 = BatchNormalization()(X21)
  X2 = layers.Maximum()([X2,X21])
  
  X3 = Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
  X3 = BatchNormalization()(X3)
  X31 =  Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
  X31 = BatchNormalization()(X31)
  X = layers.Maximum()([X3,X31])
  X = Conv2D(64,(2,2),strides=(1,1),padding='valid',activation='relu')(X)
  
  X = Concatenate()([X2,X])
  #X = Conv2D(5,(21,21),strides=(1,1))(X)
  #X = Activation('softmax')(X)
  
  #model = Model(inputs = X_input, outputs = X)
  return X

In [0]:
def input_cascade(input_shape1,input_shape2):
  
  X1_input = Input(input_shape1)
  X1 = two_path(X1_input)
  X1 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X1)
  X1 = BatchNormalization()(X1)
  
  X2_input = Input(input_shape2)
  X2_input1 = Concatenate()([X1,X2_input])
  #X2_input1 = Input(tensor = X2_input1)
  X2 = two_path(X2_input1)
  X2 = Conv2D(5,(21,21),strides=(1,1),padding='valid')(X2)
  X2 = BatchNormalization()(X2)
  X2 = Activation('softmax')(X2)
  
  model = Model(inputs=[X1_input,X2_input],outputs=X2)
  return model
  

In [0]:
def MFCcascade(input_shape1,input_shape2):
  
  X1_input = Input(input_shape1)
  X1 = two_path(X1_input)
  X1 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X1)
  X1 = BatchNormalization()(X1)
  #X1 = MaxPooling2D((2,2))(X1)
  
  X2_input = Input(input_shape2)
  X2 = two_path(X2_input)
  
  X2 = Concatenate()([X1,X2])
  X2 = Conv2D(5,(21,21),strides=(1,1),padding='valid',activation='relu')(X2)
  X2 = BatchNormalization()(X2)
  X2 = Activation('softmax')(X2)
  
  model = Model(inputs=[X1_input,X2_input],outputs=X2)
  return model
  

In [0]:
m = MFCcascade((53,53,4),(33,33,4))
m.summary()

In [0]:
m.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
m.save('trial_0001_MFCcascade_acc.h5')

In [0]:
m1 = input_cascade((65,65,4),(33,33,4))
m1.summary()

In [0]:
import os
os.chdir('drive/brat')

In [0]:
def model_gen(input_dim,x,y,slice_no):
  X1 = []
  X2 = []
  Y = []
  print(int((input_dim)/2))
  for i in range(int((input_dim)/2),175-int((input_dim)/2)):
    for j in range(int((input_dim)/2),195-int((input_dim)/2)):
      X2.append(x[i-16:i+17,j-16:j+17,:])
      X1.append(x[i-int((input_dim)/2):i+int((input_dim)/2)+1,j-int((input_dim)/2):j+int((input_dim)/2)+1,:])
      Y.append(y[i,slice_no,j])
      
  X1 = np.asarray(X1)
  X2 = np.asarray(X2)
  Y = np.asarray(Y)
  d = [X1,X2,Y]
  return d

In [0]:
def data_gen(path,slice_no,model_no):
  p = os.listdir(path)
  p.sort(key=str.lower)
  arr = []
  for i in range(len(p)):
    if(i != 4):
      p1 = os.listdir(path+'/'+p[i])
      p1.sort()
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
      arr.append(sitk.GetArrayFromImage(img))
    else:
      p1 = os.listdir(path+'/'+p[i])
      img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
      y = sitk.GetArrayFromImage(img)    
  data = np.zeros((196,176,216,4))
  for i in range(196):
    data[i,:,:,0] = arr[0][:,i,:]
    data[i,:,:,1] = arr[1][:,i,:]
    data[i,:,:,2] = arr[2][:,i,:]
    data[i,:,:,3] = arr[3][:,i,:]
  x = data[slice_no]
  
  if(model_no == 0):
    X1 = []
    for i in range(16,159):
      for j in range(16,199):
        X1.append(x[i-16:i+17,j-16:j+17,:])
    Y1 = []
    for i in range(16,159):
      for j in range(16,199):
        Y1.append(y[i,slice_no,j]) 
    X1 = np.asarray(X1)
    Y1 = np.asarray(Y1)
    d = [X1,Y1]
  elif(model_no == 1):
    d = model_gen(65,x,y,slice_no)
  elif(model_no == 2):
    d = model_gen(56,x,y,slice_no)
  elif(model_no == 3):
    d = model_gen(53,x,y,slice_no)  
    
  return d   

In [0]:
d = data_gen('LG/0001',100,3)

In [0]:
d[2].all == 0

In [0]:
len(d[0])

In [0]:
!pip3 install SimpleITK

In [0]:
import SimpleITK as sitk
import numpy as np

In [0]:
y = np.zeros((17589,1,1,5))

In [0]:
for i in range(y.shape[0]):
  y[i,:,:,d[2][i]] = 1

In [0]:
sample = np.zeros((5,1))
for i in range(5):
  sample[i] = np.sum(y[:,:,:,i])
print(sample/np.sum(sample))  

In [0]:
X1 = np.asarray(d[0])

In [0]:
X1.shape

In [0]:
X2 = np.asarray(d[1])

In [0]:
X2.shape

In [0]:
m1.inputs

In [0]:
m.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[f1_score])

In [0]:
m_info = m.fit([X1,X2],y,epochs=20,batch_size=256)

In [0]:
m.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Slice 136 patient 0002

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(d[2]),
                                                 d[2])

In [0]:
class_weights

In [0]:
import keras
model = keras.models.load_model('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
m_info = m.fit([X1,X2],y,epochs= 20,batch_size = 256,class_weight = class_weights)

In [0]:
import matplotlib.pyplot as plt
plt.plot(m_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
m.save('trial_MFCcascade_acc.h5')

eval on 128th slice 0002

In [0]:
model.evaluate([X1,X2],y,batch_size = 1024)

In [0]:
model_info = model.fit([X1,X2],y,epochs=30,batch_size=256,class_weight= class_weights)

In [0]:
import matplotlib.pyplot as plt
plt.plot(model_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
model.save('trial_0001_MFCcas_dim2_128_acc.h5')

eval on 100th slice 0001

In [0]:
model.evaluate([X1,X2],y,batch_size = 1024)

In [0]:
pred = model.predict([X1,X2],batch_size = 1024)
pred = np.around(pred)
pred1 = np.dot(pred.reshape(17589,5),np.array([0,1,2,3,4]))
y1 = np.dot(y.reshape(17589,5),np.array([0,1,2,3,4]))

In [0]:
y2 = np.argmax(y.reshape(17589,5),axis = 1)
y2.all() == 0

In [0]:
y1.all()==0

In [0]:
from sklearn import metrics

In [0]:
f1 = metrics.f1_score(y1,pred1,average='micro')
f1

In [0]:
p1 = metrics.precision_score(y1,pred1,average='micro')
p1

In [0]:
r1 = metrics.recall_score(y1,pred1,average='micro')
r1

In [0]:
p2 = metrics.precision_score(y1,pred2,average='micro')
p2

In [0]:
pred2 = np.zeros((17589))
f2 = metrics.f1_score(y1,pred2,average='micro')
f2

Slice 128 patient 0001

In [0]:
from sklearn.utils import class_weight

In [0]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(d[2]),
                                                 d[2])

In [0]:
class_weights

In [0]:
m1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
m1_info = m1.fit([X1,X2],y,epochs=20,batch_size=256,class_weight= class_weights)

plot of inputcascade

In [0]:
import matplotlib.pyplot as plt

In [0]:
plt.plot(m1_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
m1.save('trial_0001_input_cascade_acc.h5')

In [0]:
plt.plot(m_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Training on slice 128, evaluating on 136

In [0]:
m.evaluate([X1,X2],y,batch_size = 1024)

In [0]:
m.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
pred = m.predict([X1,X2],batch_size = 1024)

In [0]:
print(((pred != 0.) & (pred != 1.)).any())

In [0]:
pred = np.around(pred)

In [0]:
type(y)

In [0]:
pred1 = np.dot(pred.reshape(17589,5),np.array([0,1,2,3,4]))

In [0]:
pred1.shape

In [0]:
y1 = np.dot(y.reshape(17589,5),np.array([0,1,2,3,4]))

In [0]:
from sklearn import metrics

In [0]:
f1 = metrics.f1_score(y1,pred1,average='micro')
f1

In [0]:
pred2 = np.zeros((17589,1))
f1 = f1 = metrics.f1_score(y1,pred2,average='micro')
f1

In [0]:
f1 = metrics.f1_score(y1,pred1,average='weighted')
f1

In [0]:
f1 = metrics.f1_score(y1,pred1,average='macro')
f1

In [0]:
plt.plot(m_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
m_info = m.fit([X1,X2],y,epochs=20,batch_size=256,class_weight= 10*class_weights)

In [0]:
plt.plot(m_info.history['acc'])
#plt.plot(m_info.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
import matplotlib.pyplot as plt

In [0]:
m.save('trial_0001_MFCcas_dim2_128_acc.h5')

In [0]:
import keras

In [0]:
def two_pathcnn(input_shape):
  
  X_input = Input(input_shape)
  
  X = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
  X = BatchNormalization()(X)
  X1 = Conv2D(64,(7,7),strides=(1,1),padding='valid')(X_input)
  X1 = BatchNormalization()(X1)
  X = layers.Maximum()([X,X1])
  X = Conv2D(64,(4,4),strides=(1,1),padding='valid',activation='relu')(X)
  
  X2 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
  X2 = BatchNormalization()(X2)
  X21 = Conv2D(160,(13,13),strides=(1,1),padding='valid')(X_input)
  X21 = BatchNormalization()(X21)
  X2 = layers.Maximum()([X2,X21])
  
  X3 = Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
  X3 = BatchNormalization()(X3)
  X31 =  Conv2D(64,(3,3),strides=(1,1),padding='valid')(X)
  X31 = BatchNormalization()(X31)
  X = layers.Maximum()([X3,X31])
  X = Conv2D(64,(2,2),strides=(1,1),padding='valid',activation='relu')(X)
  
  X = Concatenate()([X2,X])
  X = Conv2D(5,(21,21),strides=(1,1),padding='valid')(X)
  X = Activation('softmax')(X)
  
  model = Model(inputs = X_input, outputs = X)
  return model 

In [0]:
import os

In [0]:
m0 = two_pathcnn((33,33,4))
m0.summary()

In [0]:
os.chdir('drive/brat')

In [0]:
!ls

for training over entire image, create batch of patches for one image, batch of labels in Y

In [0]:
import h5py
import numpy as np

In [0]:
hf = h5py.File('data_trial_dim2_128.h5', 'r')
X = hf.get('dataset_1')
Y = hf.get('dataset_2')

In [0]:
y = np.zeros((26169,1,1,5))

In [0]:
for i in range(y.shape[0]):
  y[i,:,:,Y[i]] = 1

In [0]:
X = np.asarray(X)

In [0]:
X.shape

In [0]:
keras.__version__

In [0]:
import keras.backend as K

def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_true, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_pred, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [0]:
from sklearn.utils import class_weight

In [0]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(Y),
                                                 Y)

In [0]:
m0.compile(optimizer='adam',loss='categorical_hinge',metrics=[f1_score])

In [0]:
m0_info = m0.fit(X,y,epochs=20,batch_size=1024,class_weight = class_weights)

In [0]:
m0.save('trial_0001_dim2_128_f1.h5')

In [0]:
!ls

In [0]:
m0.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
m0_info = m0.fit(X,y,epochs=20,batch_size=4096,class_weight = class_weights)

In [0]:
m0.save('trial_0001_dim2_128_accuracy.h5')

In [0]:
!ls

In [0]:
mod = keras.models.load_model('trial_0001_81_accuracy.h5')

In [0]:
mod.evaluate(X,y,batch_size = 1024)

In [0]:
pred = m0.predict(X,batch_size = 1024)

In [0]:
pred.shape

In [0]:
pred = np.floor(pred)

In [0]:
y.reshape(26169,5)

In [0]:
pred.astype(int)

In [0]:
pred = pred.reshape(26169,5)
y_pred = np.floor(np.dot(pred,np.array([0,1,2,3,4])))
y_pred.reshape(26169,1)

In [0]:
y_pred.shape

In [0]:
print(((y_pred != 0.) & (y_pred != 1.)).any())

In [0]:
from matplotlib import pyplot as plt
plt.imshow(np.uint8(y_pred*32))
plt.show()

In [0]:
from sklearn import metrics

In [0]:
f1 = metrics.f1_score(y,pred)

In [0]:
!pip3 install SimpleITK

In [0]:
import SimpleITK as sitk
import numpy as np

In [0]:
path = 'LG/0001'
p = os.listdir(path)
p.sort(key=str.lower)
arr = []
for i in range(len(p)):
  if(i != 4):
    p1 = os.listdir(path+'/'+p[i])
    p1.sort()
    img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[-1])
    arr.append(sitk.GetArrayFromImage(img))
  else:
    p1 = os.listdir(path+'/'+p[i])
    img = sitk.ReadImage(path+'/'+p[i]+'/'+p1[0])
    Y_labels = sitk.GetArrayFromImage(img)    
data = np.zeros((Y_labels.shape[1],Y_labels.shape[0],Y_labels.shape[2],4))
for i in range(196):
  data[i,:,:,0] = arr[0][:,i,:]
  data[i,:,:,1] = arr[1][:,i,:]
  data[i,:,:,2] = arr[2][:,i,:]
  data[i,:,:,3] = arr[3][:,i,:]
  

In [0]:
def model_gen(input_dim,x,y,slice_no):
  X1 = []
  X2 = []
  Y = []
  
  for i in range(int((input_dim)/2),175-int((input_dim)/2)):
    for j in range(int((input_dim)/2),195-int((input_dim)/2)):
      if(x[i-16:i+17,j-16:j+17,:].any != 0):
        X2.append(x[i-16:i+17,j-16:j+17,:])
        X1.append(x[i-int((input_dim)/2):i+int((input_dim)/2)+1,j-int((input_dim)/2):j+int((input_dim)/2)+1,:])
        Y.append(y[i,slice_no,j])
      
      
  X1 = np.asarray(X1)
  X2 = np.asarray(X2)
  Y = np.asarray(Y)
  d = [X1,X2,Y]
  return d

In [0]:
def data_gen(data,y,slice_no,model_no):
  d = []
  x = data[slice_no]
  if(x.any() != 0 and y.any() != 0):
    if(model_no == 0):
      X1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            X1.append(x[i-16:i+17,j-16:j+17,:])
      Y1 = []
      for i in range(16,159):
        for j in range(16,199):
          if(x[i-16:i+17,j-16:j+17,:].all != 0):
            Y1.append(y[i,slice_no,j]) 
      X1 = np.asarray(X1)
      Y1 = np.asarray(Y1)
      d = [X1,Y1]
    elif(model_no == 1):
      d = model_gen(65,x,y,slice_no)
    elif(model_no == 2):
      d = model_gen(56,x,y,slice_no)
    elif(model_no == 3):
      d = model_gen(53,x,y,slice_no)  
    
  return d   

In [0]:
from sklearn.utils import class_weight

In [0]:
m0.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
info = []
for i in range(90,data.shape[0],2):
  d = data_gen(data,Y_labels,i,0)
  if(len(d) != 0):
    y = np.zeros((d[-1].shape[0],1,1,5))
    for j in range(y.shape[0]):
      y[j,:,:,d[-1][j]] = 1
    X1 = d[0]
    class_weights = class_weight.compute_class_weight('balanced',
                                                      np.unique(d[-1]),
                                                      d[-1])
    print('slice no:'+str(i))
    info.append(m0.fit(X1,y,epochs=2,batch_size=32,class_weight= class_weights))
    m0.save('trial_0001_2path_acc.h5')